In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('dataset/24-hours testset - Copy.csv')
dataset.head()

,timestamp,day_of_week,hour_of_day,is_weekend,temperature,is_holiday,solar_generation,electricity_demand
0,2024-01-01 00:00:00,0,0,0,28.534249,0,0.786249,633.109944
1,2024-01-01 01:00:00,0,1,0,18.140664,0,7.331477,537.356804
2,2024-01-01 02:00:00,0,2,0,19.539847,0,9.877753,562.069786
3,2024-01-01 03:00:00,0,3,0,17.519129,0,1.215103,483.268876
4,2024-01-01 04:00:00,0,4,0,18.255590,0,2.476539,579.301366


In [3]:
from meteostat import Stations, Hourly
from datetime import datetime

start = datetime(2024, 1, 1)
end = datetime(2024, 12, 31)

stations=Stations()
delhi = stations.region('IN', 'DL').fetch(1)  # Fetch the top station in Delhi
delhi_station_id=delhi.index[0]

In [ ]:
data=Hourly(delhi_station_id, start, end)
data = data.fetch()

In [5]:
data.to_csv('dataset/weather_data_delhi_2024.csv')
print('Data saved to dataset/weather_data_delhi_2024.csv')

Data saved to dataset/weather_data_delhi_2024.csv


In [6]:
weather_2024=pd.read_csv('dataset/weather_data_delhi_2024.csv')
weather_2024.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2024-01-01 00:00:00,12.2,9.0,81.0,0.0,NaN,270.0,7.6,NaN,1019.4,NaN,5.0
1,2024-01-01 01:00:00,11.0,9.1,88.0,0.0,NaN,0.0,0.0,NaN,1019.0,NaN,5.0
2,2024-01-01 02:00:00,11.0,9.1,88.0,0.0,NaN,260.0,7.6,NaN,1020.0,NaN,5.0
3,2024-01-01 03:00:00,11.2,8.4,83.0,0.0,NaN,320.0,5.4,NaN,1021.2,NaN,5.0
4,2024-01-01 04:00:00,11.0,9.1,88.0,0.0,NaN,30.0,5.4,NaN,1021.0,NaN,5.0


In [7]:
weather_2024.isna().sum()

time       0
temp       0
dwpt       0
rhum       0
prcp     350
snow    7408
wdir     136
wspd       0
wpgt    7408
pres       0
tsun    7408
coco      17
dtype: int64

In [8]:
weather_2024 =weather_2024[['time', 'temp','dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']]
weather_2024.head()

,time,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,tsun
0,2024-01-01 00:00:00,12.2,9.0,81.0,0.0,270.0,7.6,NaN,1019.4,NaN
1,2024-01-01 01:00:00,11.0,9.1,88.0,0.0,0.0,0.0,NaN,1019.0,NaN
2,2024-01-01 02:00:00,11.0,9.1,88.0,0.0,260.0,7.6,NaN,1020.0,NaN
3,2024-01-01 03:00:00,11.2,8.4,83.0,0.0,320.0,5.4,NaN,1021.2,NaN
4,2024-01-01 04:00:00,11.0,9.1,88.0,0.0,30.0,5.4,NaN,1021.0,NaN


In [9]:
# Convert the 'time' column to datetime format
weather_2024['time'] = pd.to_datetime(weather_2024['time'])

# Set the 'time' column as the index
weather_2024.set_index('time', inplace=True)

# Define a function to fill missing values with the average of the previous 3 hours
def fill_missing_with_rolling_mean(df, window=3):
    return df.rolling(window=window, min_periods=1).mean()

# Apply the function to each column
weather_2024_filled = weather_2024.apply(lambda col: col.fillna(fill_missing_with_rolling_mean(col)))

# Reset the index
weather_2024_filled.reset_index(inplace=True)

weather_2024_filled.head()

,time,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,tsun
0,2024-01-01 00:00:00,12.2,9.0,81.0,0.0,270.0,7.6,NaN,1019.4,NaN
1,2024-01-01 01:00:00,11.0,9.1,88.0,0.0,0.0,0.0,NaN,1019.0,NaN
2,2024-01-01 02:00:00,11.0,9.1,88.0,0.0,260.0,7.6,NaN,1020.0,NaN
3,2024-01-01 03:00:00,11.2,8.4,83.0,0.0,320.0,5.4,NaN,1021.2,NaN
4,2024-01-01 04:00:00,11.0,9.1,88.0,0.0,30.0,5.4,NaN,1021.0,NaN


In [10]:
weather_2024_filled['prcp'] = weather_2024_filled['prcp'].fillna(0)
weather_2024_filled.head()

,time,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,tsun
0,2024-01-01 00:00:00,12.2,9.0,81.0,0.0,270.0,7.6,NaN,1019.4,NaN
1,2024-01-01 01:00:00,11.0,9.1,88.0,0.0,0.0,0.0,NaN,1019.0,NaN
2,2024-01-01 02:00:00,11.0,9.1,88.0,0.0,260.0,7.6,NaN,1020.0,NaN
3,2024-01-01 03:00:00,11.2,8.4,83.0,0.0,320.0,5.4,NaN,1021.2,NaN
4,2024-01-01 04:00:00,11.0,9.1,88.0,0.0,30.0,5.4,NaN,1021.0,NaN


In [11]:
weather_2024_filled.drop(columns=['tsun'], inplace=True)

In [12]:
# Define a function to determine the season based on the month
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

# Extract the month from the 'time' column
weather_2024_filled['month'] = weather_2024_filled['time'].dt.month

# Apply the function to create the 'season' column
weather_2024_filled['season'] = weather_2024_filled['month'].apply(get_season)


In [13]:
weather_2024_filled.isna().sum()

time         0
temp         0
dwpt         0
rhum         0
prcp         0
wdir        89
wspd         0
wpgt      7408
pres         0
month        0
season       0
dtype: int64

In [14]:
weather_2024_filled.drop(columns=['wpgt'], inplace=True)

In [15]:
# Calculate the average wdir for each season
seasonal_avg_wdir = weather_2024_filled.groupby('season')['wdir'].mean()

# Define a function to fill null values with the seasonal average
def fill_wdir_with_seasonal_avg(row):
    if pd.isnull(row['wdir']):
        return seasonal_avg_wdir[row['season']]
    else:
        return row['wdir']

# Apply the function to fill null values in the 'wdir' column
weather_2024_filled['wdir'] = weather_2024_filled.apply(fill_wdir_with_seasonal_avg, axis=1)

weather_2024_filled.head()

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,month,season
0,2024-01-01 00:00:00,12.2,9.0,81.0,0.0,270.0,7.6,1019.4,1,Winter
1,2024-01-01 01:00:00,11.0,9.1,88.0,0.0,0.0,0.0,1019.0,1,Winter
2,2024-01-01 02:00:00,11.0,9.1,88.0,0.0,260.0,7.6,1020.0,1,Winter
3,2024-01-01 03:00:00,11.2,8.4,83.0,0.0,320.0,5.4,1021.2,1,Winter
4,2024-01-01 04:00:00,11.0,9.1,88.0,0.0,30.0,5.4,1021.0,1,Winter


In [16]:
weather_2024_filled.tail()

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,month,season
7403,2024-11-04 11:00:00,28.8,19.2,56.0,0.0,312.000000,1.8,1011.3,11,Autumn
7404,2024-11-04 12:00:00,28.3,19.2,58.0,0.0,344.000000,1.8,1011.1,11,Autumn
7405,2024-11-04 13:00:00,26.7,19.3,64.0,0.0,328.000000,0.0,1011.5,11,Autumn
7406,2024-11-04 14:00:00,25.0,19.6,72.0,0.0,344.000000,0.0,1011.9,11,Autumn
7407,2024-11-04 15:00:00,23.4,20.0,81.0,0.0,207.900547,0.0,1012.4,11,Autumn


In [17]:
weather_2024_filled.isna().sum()

time      0
temp      0
dwpt      0
rhum      0
prcp      0
wdir      0
wspd      0
pres      0
month     0
season    0
dtype: int64

In [18]:
weather_2024_filled['time']=pd.to_datetime(weather_2024_filled['time'])

In [19]:
weather_2024_filled.rename(columns={'time': 'timestamp'}, inplace=True)

In [20]:
dataset['timestamp']=pd.to_datetime(dataset['timestamp'])

In [28]:
test_dataset=pd.merge(dataset, weather_2024_filled[['timestamp', 'dwpt', 'wspd', 'wdir', 'prcp', 'rhum', 'pres','season','month']], on='timestamp', how='left')

In [29]:
test_dataset.tail()

,timestamp,day_of_week,hour_of_day,is_weekend,temperature,is_holiday,solar_generation,electricity_demand,dwpt,wspd,wdir,prcp,rhum,pres,season,month
8756,2024-12-30 20:00:00,0,20,0,7.929691,0,4.235843,607.299343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8757,2024-12-30 21:00:00,0,21,0,10.136094,0,5.939992,551.692302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8758,2024-12-30 22:00:00,0,22,0,16.290948,0,0.673143,611.617725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8759,2024-12-30 23:00:00,0,23,0,18.676171,0,3.043134,634.507696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8760,2024-12-31 00:00:00,1,0,0,7.619222,0,7.296498,574.161026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
test_dataset.isna().sum()

timestamp                0
day_of_week              0
hour_of_day              0
is_weekend               0
temperature              0
is_holiday               0
solar_generation         0
electricity_demand       0
dwpt                  1353
wspd                  1353
wdir                  1353
prcp                  1353
rhum                  1353
pres                  1353
season                1353
month                 1353
dtype: int64

In [31]:
test_dataset.dropna(inplace=True)

In [32]:
test_dataset.tail()

,timestamp,day_of_week,hour_of_day,is_weekend,temperature,is_holiday,solar_generation,electricity_demand,dwpt,wspd,wdir,prcp,rhum,pres,season,month
7403,2024-11-04 11:00:00,0,11,0,34.906707,0,255.383277,568.931265,19.2,1.8,312.000000,0.0,56.0,1011.3,Autumn,11.0
7404,2024-11-04 12:00:00,0,12,0,34.111647,0,153.934366,473.838689,19.2,1.8,344.000000,0.0,58.0,1011.1,Autumn,11.0
7405,2024-11-04 13:00:00,0,13,0,28.932635,0,185.923463,411.604915,19.3,0.0,328.000000,0.0,64.0,1011.5,Autumn,11.0
7406,2024-11-04 14:00:00,0,14,0,31.651295,0,120.450180,589.210724,19.6,0.0,344.000000,0.0,72.0,1011.9,Autumn,11.0
7407,2024-11-04 15:00:00,0,15,0,25.624955,0,91.788827,543.378713,20.0,0.0,207.900547,0.0,81.0,1012.4,Autumn,11.0


In [33]:
test_dataset.isna().sum()

timestamp             0
day_of_week           0
hour_of_day           0
is_weekend            0
temperature           0
is_holiday            0
solar_generation      0
electricity_demand    0
dwpt                  0
wspd                  0
wdir                  0
prcp                  0
rhum                  0
pres                  0
season                0
month                 0
dtype: int64

In [34]:
test_dataset.dtypes

timestamp             datetime64[ns]
day_of_week                    int64
hour_of_day                    int64
is_weekend                     int64
temperature                  float64
is_holiday                     int64
solar_generation             float64
electricity_demand           float64
dwpt                         float64
wspd                         float64
wdir                         float64
prcp                         float64
rhum                         float64
pres                         float64
season                        object
month                        float64
dtype: object

In [40]:
test_dataset['month'].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [35]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
test_dataset['season'] = le.fit_transform(test_dataset['season'])


In [36]:
test_dataset.drop(columns=['electricity_demand'], inplace=True)

In [37]:
test_dataset.head()

,timestamp,day_of_week,hour_of_day,is_weekend,temperature,is_holiday,solar_generation,dwpt,wspd,wdir,prcp,rhum,pres,season,month
0,2024-01-01 00:00:00,0,0,0,28.534249,0,0.786249,9.0,7.6,270.0,0.0,81.0,1019.4,3,1.0
1,2024-01-01 01:00:00,0,1,0,18.140664,0,7.331477,9.1,0.0,0.0,0.0,88.0,1019.0,3,1.0
2,2024-01-01 02:00:00,0,2,0,19.539847,0,9.877753,9.1,7.6,260.0,0.0,88.0,1020.0,3,1.0
3,2024-01-01 03:00:00,0,3,0,17.519129,0,1.215103,8.4,5.4,320.0,0.0,83.0,1021.2,3,1.0
4,2024-01-01 04:00:00,0,4,0,18.255590,0,2.476539,9.1,5.4,30.0,0.0,88.0,1021.0,3,1.0


In [38]:
test_dataset.isna().sum()

timestamp           0
day_of_week         0
hour_of_day         0
is_weekend          0
temperature         0
is_holiday          0
solar_generation    0
dwpt                0
wspd                0
wdir                0
prcp                0
rhum                0
pres                0
season              0
month               0
dtype: int64

In [39]:
test_dataset.to_csv('dataset/test_dataset_2024.csv', index=False)